In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.models import Model,Sequential
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from keras.layers import GRU,Embedding,Bidirectional,InputLayer,Input,Dense,TimeDistributed,Activation,RepeatVector,Bidirectional
import os
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
print(os.listdir("../input"))
from keras.preprocessing.sequence import pad_sequences
from keras.losses import sparse_categorical_crossentropy
# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


['small_vocab_fr.txt', 'small_vocab_en.txt']


In [2]:

with open("../input/small_vocab_en.txt") as f:
    eng=f.read()

In [3]:
with open("../input/small_vocab_fr.txt") as f:
    fr=f.read()

In [4]:
fr=fr.split("\n")

In [5]:
len(fr)

137861

In [6]:
fr[:5]

["\ufeffnew jersey est parfois calme pendant l' automne , et il est neigeux en avril .",
 'les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .',
 'california est généralement calme en mars , et il est généralement chaud en juin .',
 'les états-unis est parfois légère en juin , et il fait froid en septembre .',
 'votre moins aimé fruit est le raisin , mais mon moins aimé est la pomme .']

In [7]:
eng[:5]

'new j'

In [8]:
eng=eng.split("\n")

In [9]:
eng[:5]

['new jersey is sometimes quiet during autumn , and it is snowy in april .',
 'the united states is usually chilly during july , and it is usually freezing in november .',
 'california is usually quiet during march , and it is usually hot in june .',
 'the united states is sometimes mild during june , and it is cold in september .',
 'your least liked fruit is the grape , but my least liked is the apple .']

In [10]:
len(eng)

137861

In [11]:
eng[:10]

['new jersey is sometimes quiet during autumn , and it is snowy in april .',
 'the united states is usually chilly during july , and it is usually freezing in november .',
 'california is usually quiet during march , and it is usually hot in june .',
 'the united states is sometimes mild during june , and it is cold in september .',
 'your least liked fruit is the grape , but my least liked is the apple .',
 'his favorite fruit is the orange , but my favorite is the grape .',
 'paris is relaxing during december , but it is usually chilly in july .',
 'new jersey is busy during spring , and it is never hot in march .',
 'our least liked fruit is the lemon , but my least liked is the grape .',
 'the united states is sometimes busy during january , and it is sometimes warm in november .']

In [12]:
from collections import Counter

In [13]:
engdict=Counter([token for sent in eng for token in sent.split(" ")])

In [14]:
engdict.most_common(30)

[('is', 205858),
 (',', 140897),
 ('.', 129039),
 ('in', 75525),
 ('it', 75137),
 ('during', 74933),
 ('the', 67628),
 ('but', 63987),
 ('and', 59850),
 ('sometimes', 37746),
 ('usually', 37507),
 ('never', 37500),
 ('least', 27564),
 ('favorite', 27371),
 ('fruit', 27105),
 ('most', 14934),
 ('loved', 13666),
 ('liked', 13546),
 ('new', 12197),
 ('paris', 11334),
 ('india', 11277),
 ('united', 11270),
 ('states', 11270),
 ('california', 11250),
 ('jersey', 11225),
 ('france', 11170),
 ('china', 10953),
 ('he', 10786),
 ('she', 10786),
 ('grapefruit', 10118)]

In [15]:
frdict=Counter(token for sent in fr for token in sent.split(" "))  # most common words are usually stopwords

In [16]:
frdict.most_common(20) #Most Common words are usually the stopwords

[('est', 196809),
 ('.', 135619),
 (',', 123135),
 ('en', 105768),
 ('il', 84079),
 ('les', 65255),
 ('mais', 63987),
 ('et', 59851),
 ('la', 49861),
 ('parfois', 37746),
 ('jamais', 37215),
 ('le', 35306),
 ("l'", 32917),
 ('généralement', 31292),
 ('moins', 27557),
 ('au', 25738),
 ('aimé', 24842),
 ('fruit', 23626),
 ('préféré', 22886),
 ('agréable', 17751)]

In [17]:
eng_vocabsize=len(engdict.keys())

In [18]:
fr_vocabsize=len(frdict.keys())

In [19]:
eng_vocabsize

228

In [20]:
fr_vocabsize

357

In [21]:
#Tokenize words into indexes we use keras tokenizer
engtokenizer=Tokenizer()
engtokenizer.fit_on_texts(eng)


In [22]:
eng_token=engtokenizer.texts_to_sequences(eng)

In [23]:
eng_token[:5]

[[17, 23, 1, 8, 67, 4, 39, 7, 3, 1, 55, 2, 44],
 [5, 20, 21, 1, 9, 62, 4, 43, 7, 3, 1, 9, 51, 2, 45],
 [22, 1, 9, 67, 4, 38, 7, 3, 1, 9, 68, 2, 34],
 [5, 20, 21, 1, 8, 64, 4, 34, 7, 3, 1, 57, 2, 42],
 [29, 12, 16, 13, 1, 5, 82, 6, 30, 12, 16, 1, 5, 83]]

In [24]:
frtokenizer=Tokenizer()
frtokenizer.fit_on_texts(fr)
fr_token=frtokenizer.texts_to_sequences(fr)

In [25]:
fr[:5]

["\ufeffnew jersey est parfois calme pendant l' automne , et il est neigeux en avril .",
 'les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .',
 'california est généralement calme en mars , et il est généralement chaud en juin .',
 'les états-unis est parfois légère en juin , et il fait froid en septembre .',
 'votre moins aimé fruit est le raisin , mais mon moins aimé est la pomme .']

In [26]:
fr_token[:5]

[[337, 34, 1, 8, 67, 37, 11, 24, 6, 3, 1, 112, 2, 50],
 [4, 32, 31, 1, 12, 19, 2, 49, 6, 3, 95, 69, 2, 51],
 [101, 1, 12, 67, 2, 45, 6, 3, 1, 12, 21, 2, 41],
 [4, 32, 31, 1, 8, 269, 2, 41, 6, 3, 103, 19, 2, 48],
 [40, 13, 14, 16, 1, 10, 82, 5, 39, 13, 14, 1, 7, 83]]

In [27]:
eng_ind=engtokenizer.word_index

In [28]:
eng_ind

{'is': 1,
 'in': 2,
 'it': 3,
 'during': 4,
 'the': 5,
 'but': 6,
 'and': 7,
 'sometimes': 8,
 'usually': 9,
 'never': 10,
 'favorite': 11,
 'least': 12,
 'fruit': 13,
 'most': 14,
 'loved': 15,
 'liked': 16,
 'new': 17,
 'paris': 18,
 'india': 19,
 'united': 20,
 'states': 21,
 'california': 22,
 'jersey': 23,
 'france': 24,
 'china': 25,
 'he': 26,
 'she': 27,
 'grapefruit': 28,
 'your': 29,
 'my': 30,
 'his': 31,
 'her': 32,
 'fall': 33,
 'june': 34,
 'spring': 35,
 'january': 36,
 'winter': 37,
 'march': 38,
 'autumn': 39,
 'may': 40,
 'nice': 41,
 'september': 42,
 'july': 43,
 'april': 44,
 'november': 45,
 'summer': 46,
 'december': 47,
 'february': 48,
 'our': 49,
 'their': 50,
 'freezing': 51,
 'pleasant': 52,
 'beautiful': 53,
 'october': 54,
 'snowy': 55,
 'warm': 56,
 'cold': 57,
 'wonderful': 58,
 'dry': 59,
 'busy': 60,
 'august': 61,
 'chilly': 62,
 'rainy': 63,
 'mild': 64,
 'wet': 65,
 'relaxing': 66,
 'quiet': 67,
 'hot': 68,
 'dislikes': 69,
 'likes': 70,
 'limes': 7

In [29]:
fr_ind=frtokenizer.word_index

In [30]:
fr_ind

{'est': 1,
 'en': 2,
 'il': 3,
 'les': 4,
 'mais': 5,
 'et': 6,
 'la': 7,
 'parfois': 8,
 'jamais': 9,
 'le': 10,
 "l'": 11,
 'généralement': 12,
 'moins': 13,
 'aimé': 14,
 'au': 15,
 'fruit': 16,
 'préféré': 17,
 'agréable': 18,
 'froid': 19,
 'son': 20,
 'chaud': 21,
 'de': 22,
 'plus': 23,
 'automne': 24,
 'mois': 25,
 'à': 26,
 'elle': 27,
 'citrons': 28,
 'paris': 29,
 'inde': 30,
 'unis': 31,
 'états': 32,
 'france': 33,
 'jersey': 34,
 'new': 35,
 'chine': 36,
 'pendant': 37,
 'pamplemousse': 38,
 'mon': 39,
 'votre': 40,
 'juin': 41,
 'printemps': 42,
 'janvier': 43,
 'hiver': 44,
 'mars': 45,
 'été': 46,
 'mai': 47,
 'septembre': 48,
 'juillet': 49,
 'avril': 50,
 'novembre': 51,
 'décembre': 52,
 'février': 53,
 'octobre': 54,
 'aime': 55,
 'août': 56,
 'merveilleux': 57,
 'relaxant': 58,
 'doux': 59,
 'humide': 60,
 'notre': 61,
 'californie': 62,
 'sec': 63,
 'leur': 64,
 'occupé': 65,
 'pluvieux': 66,
 'calme': 67,
 'beau': 68,
 'habituellement': 69,
 'pommes': 70,
 'pêch

In [31]:
eng_token_pad=pad_sequences(eng_token,padding='post')

In [32]:
fr_token_pad=pad_sequences(fr_token,padding='post')

In [33]:
eng_token[:5]

[[17, 23, 1, 8, 67, 4, 39, 7, 3, 1, 55, 2, 44],
 [5, 20, 21, 1, 9, 62, 4, 43, 7, 3, 1, 9, 51, 2, 45],
 [22, 1, 9, 67, 4, 38, 7, 3, 1, 9, 68, 2, 34],
 [5, 20, 21, 1, 8, 64, 4, 34, 7, 3, 1, 57, 2, 42],
 [29, 12, 16, 13, 1, 5, 82, 6, 30, 12, 16, 1, 5, 83]]

In [34]:
eng_token_pad[:5].shape

(5, 15)

In [35]:
# 15 is max english length size

In [36]:
eng_token_pad.shape

(137861, 15)

In [37]:
fr_token_pad.shape

(137861, 21)

In [38]:
# 21 is max fr token size

In [39]:
eng_token_pad=eng_token_pad.reshape(*eng_token_pad.shape,1)
fr_token_pad=fr_token_pad.reshape(*fr_token_pad.shape,1)

In [40]:
eng_token_pad.shape

(137861, 15, 1)

In [41]:
ind_w=frtokenizer.index_word

In [42]:
ind_w[0]='<PAD>'

In [43]:
max_eng_length=eng_token_pad.shape[1]
max_fr_length=fr_token_pad.shape[1]

In [44]:
max_eng_length

15

In [45]:
max_fr_length

21

In [46]:
#Here we Implement Four Different Models


**Model 1**

In [47]:
inp=Input(shape=[21,1])
x=GRU(64,return_sequences=True)(inp)
logits=TimeDistributed(Dense(fr_vocabsize))(x)
model=Model(inp,Activation('softmax')(logits))

Instructions for updating:
Colocations handled automatically by placer.


In [48]:
eng_token[:5]

[[17, 23, 1, 8, 67, 4, 39, 7, 3, 1, 55, 2, 44],
 [5, 20, 21, 1, 9, 62, 4, 43, 7, 3, 1, 9, 51, 2, 45],
 [22, 1, 9, 67, 4, 38, 7, 3, 1, 9, 68, 2, 34],
 [5, 20, 21, 1, 8, 64, 4, 34, 7, 3, 1, 57, 2, 42],
 [29, 12, 16, 13, 1, 5, 82, 6, 30, 12, 16, 1, 5, 83]]

In [49]:
tmp_x=pad_sequences(eng_token,maxlen=max_fr_length,padding='post')

In [50]:
tmp_x=tmp_x.reshape((-1,21,1))

In [51]:
model.compile(loss=sparse_categorical_crossentropy,optimizer='adam')

In [52]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 21, 1)             0         
_________________________________________________________________
gru_1 (GRU)                  (None, 21, 64)            12672     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 21, 357)           23205     
_________________________________________________________________
activation_1 (Activation)    (None, 21, 357)           0         
Total params: 35,877
Trainable params: 35,877
Non-trainable params: 0
_________________________________________________________________


In [53]:
tmp_x.shape

(137861, 21, 1)

In [54]:
fr_token_pad.shape[-2]

21

In [55]:
# https://jovianlin.io/cat-crossentropy-vs-sparse-cat-crossentropy/

In [56]:
tmp_x.shape

(137861, 21, 1)

In [57]:
fr_token_pad.shape

(137861, 21, 1)

In [58]:
model.fit(tmp_x,fr_token_pad,batch_size=1024,epochs=30,validation_split=0.2)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 110288 samples, validate on 27573 samples
Epoch 1/30
110288/110288 [==============================] - 7s 59us/step - loss: 3.5216 - val_loss: 2.5643
Epoch 2/30
110288/110288 [==============================] - 4s 39us/step - loss: 2.4506 - val_loss: 2.3328
Epoch 3/30
110288/110288 [==============================] - 4s 40us/step - loss: 2.2141 - val_loss: 2.0862
Epoch 4/30
110288/110288 [==============================] - 4s 39us/step - loss: 1.9708 - val_loss: 1.8655
Epoch 5/30
110288/110288 [==============================] - 4s 39us/step - loss: 1.7952 - val_loss: 1.7350
Epoch 6/30
110288/110288 [==============================] - 4s 39us/step - loss: 1.6933 - val_loss: 1.6549
Epoch 7/30
110288/110288 [==============================] - 4s 39us/step - loss: 1.6240 - val_loss: 1.5932
Epoch 8/30
110288/110288 [==============================] - 4s 39us/step -

In [59]:
text1=["i am very quiet during spring and autumn"]

text1=engtokenizer.texts_to_sequences(text1)

In [60]:
a=pad_sequences(text1,maxlen=max_fr_length,padding='post')

In [61]:
a=a.reshape((-1,21,1))

In [62]:
pred_of_a=model.predict(a)
pred=np.argmax(pred_of_a,axis=2)

In [63]:
" ".join([ind_w[i] for i in pred[0]])

'ils vous les les et et et et <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>'

In [64]:
pred[0]

array([ 99, 106,   4,   4,   6,   6,   6,   6,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0])

In [65]:
from sklearn.metrics import accuracy_score

In [66]:
pred=model.predict(tmp_x)

In [67]:
pred.shape

(137861, 21, 357)

In [68]:
pred=np.argmax(pred,axis=-1)

In [69]:
pred=pred.reshape((-1,1))

In [70]:
pred.shape

(2895081, 1)

In [71]:
m=fr_token_pad

In [72]:
m=m.reshape((-1,1))

In [73]:
acc=accuracy_score(pred,m)

In [74]:
acc # so we are getting 61% accuracy with 100 training examples

0.6570924958576289

**Model2**

In [75]:
max_fr_length

21

In [76]:
model=Sequential()
model.add(InputLayer((21,)))
model.add(Embedding(eng_vocabsize,64,input_length=max_fr_length))
model.add(GRU(64,return_sequences=True,activation='tanh'))
model.add(TimeDistributed(Dense(fr_vocabsize,activation='softmax')))
model.compile(loss=sparse_categorical_crossentropy,optimizer='adam')

In [77]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 21, 64)            14592     
_________________________________________________________________
gru_2 (GRU)                  (None, 21, 64)            24768     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 21, 357)           23205     
Total params: 62,565
Trainable params: 62,565
Non-trainable params: 0
_________________________________________________________________


In [78]:
tmp_x=tmp_x.reshape((-1,21))

In [79]:
model.fit(tmp_x,fr_token_pad,batch_size=1024,epochs=30,verbose=1,validation_split=0.2)

Train on 110288 samples, validate on 27573 samples
Epoch 1/30
110288/110288 [==============================] - 5s 49us/step - loss: 3.7871 - val_loss: 2.9764
Epoch 2/30
110288/110288 [==============================] - 5s 41us/step - loss: 2.6719 - val_loss: 2.3356
Epoch 3/30
110288/110288 [==============================] - 5s 42us/step - loss: 1.9505 - val_loss: 1.6214
Epoch 4/30
110288/110288 [==============================] - 5s 42us/step - loss: 1.4182 - val_loss: 1.2434
Epoch 5/30
110288/110288 [==============================] - 5s 42us/step - loss: 1.1255 - val_loss: 1.0209
Epoch 6/30
110288/110288 [==============================] - 5s 42us/step - loss: 0.9452 - val_loss: 0.8752
Epoch 7/30
110288/110288 [==============================] - 5s 42us/step - loss: 0.8216 - val_loss: 0.7692
Epoch 8/30
110288/110288 [==============================] - 5s 42us/step - loss: 0.7316 - val_loss: 0.6915
Epoch 9/30
110288/110288 [==============================] - 5s 42us/step - loss: 0.6620 - val

In [80]:
# Test it out
text1=["i am very quiet during spring and autumn"]
text1=engtokenizer.texts_to_sequences(text1)
a=pad_sequences(text1,maxlen=max_fr_length,padding='post')
a=a.reshape((-1,21))
pred_of_a=model.predict(a)
pred=np.argmax(pred_of_a,axis=2)

In [81]:
" ".join([ind_w[i] for i in pred[0]])

'je je je calme calme tranquille je calme <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>'

In [82]:
pred=model.predict(tmp_x)

In [83]:
pred=np.argmax(pred,axis=-1)

In [84]:
pred=pred.reshape((-1,1))

In [85]:
fr_token_pad1=fr_token_pad.reshape((-1,1))

In [86]:
acc=accuracy_score(pred,fr_token_pad1)

In [87]:
acc

0.9071393857373938

**Model 3**

**Bidirectional GRU Architecture**

In [88]:
model=Sequential()
model.add(InputLayer((21,)))
model.add(Embedding(eng_vocabsize,64,input_length=max_fr_length))
model.add(Bidirectional(GRU(64,return_sequences=True,activation='tanh')))
model.add(TimeDistributed(Dense(fr_vocabsize,activation='softmax')))
model.compile(loss=sparse_categorical_crossentropy,optimizer='adam')

In [89]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 21, 64)            14592     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 21, 128)           49536     
_________________________________________________________________
time_distributed_3 (TimeDist (None, 21, 357)           46053     
Total params: 110,181
Trainable params: 110,181
Non-trainable params: 0
_________________________________________________________________


In [90]:
#hidden states of GRU are concatenated

In [91]:
model.fit(tmp_x,fr_token_pad,batch_size=1024,epochs=30,verbose=1,validation_split=0.2)

Train on 110288 samples, validate on 27573 samples
Epoch 1/30
110288/110288 [==============================] - 10s 89us/step - loss: 3.4524 - val_loss: 2.5207
Epoch 2/30
110288/110288 [==============================] - 8s 76us/step - loss: 2.2622 - val_loss: 1.8437
Epoch 3/30
110288/110288 [==============================] - 8s 75us/step - loss: 1.5126 - val_loss: 1.2588
Epoch 4/30
110288/110288 [==============================] - 8s 75us/step - loss: 1.1093 - val_loss: 0.9850
Epoch 5/30
110288/110288 [==============================] - 9s 78us/step - loss: 0.9000 - val_loss: 0.8208
Epoch 6/30
110288/110288 [==============================] - 8s 75us/step - loss: 0.7609 - val_loss: 0.7022
Epoch 7/30
110288/110288 [==============================] - 8s 75us/step - loss: 0.6551 - val_loss: 0.6098
Epoch 8/30
110288/110288 [==============================] - 8s 77us/step - loss: 0.5708 - val_loss: 0.5341
Epoch 9/30
110288/110288 [==============================] - 8s 77us/step - loss: 0.5049 - va

In [92]:
# Test it out
text1=["i am very quiet during spring and autumn"]
text1=engtokenizer.texts_to_sequences(text1)
a=pad_sequences(text1,maxlen=max_fr_length,padding='post')
a=a.reshape((-1,21))
pred_of_a=model.predict(a)
pred=np.argmax(pred_of_a,axis=2)

In [93]:
" ".join([ind_w[i] for i in pred[0]])

"j'aime il calme calme au printemps et <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>"

In [94]:
pred=model.predict(tmp_x)

In [95]:
pred=np.argmax(pred,axis=-1)
pred=pred.reshape((-1,1))

In [96]:
acc=accuracy_score(pred,fr_token_pad1)

In [97]:
acc 

0.9534382630399633

In [98]:
#Reason for Using GRU  instead of LSTM because Sentences are short .GRU is fast as compared to lstm and less complicated

**Model 4**
**Seq2Seq Architecture**

In [99]:
model=Sequential()
model.add(GRU(128,input_shape=(21,1)))
model.add(RepeatVector(max_fr_length))
model.add(GRU(128,return_sequences=True))
model.add(TimeDistributed(Dense(fr_vocabsize,activation='softmax')))
model.compile(loss=sparse_categorical_crossentropy,optimizer='adam')

In [100]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_4 (GRU)                  (None, 128)               49920     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 21, 128)           0         
_________________________________________________________________
gru_5 (GRU)                  (None, 21, 128)           98688     
_________________________________________________________________
time_distributed_4 (TimeDist (None, 21, 357)           46053     
Total params: 194,661
Trainable params: 194,661
Non-trainable params: 0
_________________________________________________________________


In [101]:
#This is different way of implementing Encoder Decoder as in tensorflow
# in tensorflow . last encoder hidden state is given as initial hidden state of 
# decoder and predictions are made
# here iniital state is zero for decoder but last hidden state of decoder 
# is given as input for in form of timesteps for decoder


In [102]:
tmp_x=tmp_x.reshape((-1,21,1))

In [103]:
model.fit(tmp_x,fr_token_pad,batch_size=1024,epochs=30,verbose=1,validation_split=0.2)

Train on 110288 samples, validate on 27573 samples
Epoch 1/30
110288/110288 [==============================] - 10s 91us/step - loss: 3.0970 - val_loss: 2.4964
Epoch 2/30
110288/110288 [==============================] - 8s 76us/step - loss: 2.3625 - val_loss: 2.2523
Epoch 3/30
110288/110288 [==============================] - 8s 75us/step - loss: 2.1834 - val_loss: 2.1157
Epoch 4/30
110288/110288 [==============================] - 8s 75us/step - loss: 2.0269 - val_loss: 1.8955
Epoch 5/30
110288/110288 [==============================] - 8s 75us/step - loss: 1.7829 - val_loss: 1.7033
Epoch 6/30
110288/110288 [==============================] - 8s 76us/step - loss: 1.6488 - val_loss: 1.5984
Epoch 7/30
110288/110288 [==============================] - 8s 75us/step - loss: 1.5652 - val_loss: 1.5243
Epoch 8/30
110288/110288 [==============================] - 8s 75us/step - loss: 1.4959 - val_loss: 1.4646
Epoch 9/30
110288/110288 [==============================] - 8s 75us/step - loss: 1.4425 - va

In [104]:
# Test it out
text1=["i am very quiet during spring and autumn"]
text1=engtokenizer.texts_to_sequences(text1)
a=pad_sequences(text1,maxlen=max_fr_length,padding='post')
a=a.reshape((-1,21,1))
pred_of_a=model.predict(a)
pred=np.argmax(pred_of_a,axis=2)

In [105]:
" ".join([ind_w[i] for i in pred[0]])

'ils aiment le visiter le citrons et <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>'

In [106]:
pred=model.predict(tmp_x)

In [107]:
pred=np.argmax(pred,axis=-1)
pred=pred.reshape((-1,1))

In [108]:
acc=accuracy_score(pred,fr_token_pad1)

In [109]:
acc 

0.6828258000380646

**Model 5  Seq2seq with Embeddings**

In [110]:
model=Sequential()
model.add(InputLayer((21,)))
model.add(Embedding(eng_vocabsize,32,input_length=21))
model.add(Bidirectional(GRU(256,activation='tanh')))
model.add(RepeatVector(max_fr_length))
model.add(GRU(256,return_sequences=True))
model.add(TimeDistributed(Dense(fr_vocabsize,activation='softmax')))
model.compile(loss=sparse_categorical_crossentropy,optimizer='adam')

In [111]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 21, 32)            7296      
_________________________________________________________________
bidirectional_2 (Bidirection (None, 512)               443904    
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 21, 512)           0         
_________________________________________________________________
gru_7 (GRU)                  (None, 21, 256)           590592    
_________________________________________________________________
time_distributed_5 (TimeDist (None, 21, 357)           91749     
Total params: 1,133,541
Trainable params: 1,133,541
Non-trainable params: 0
_________________________________________________________________


In [112]:
tmp_x=tmp_x.reshape((-1,21))

In [113]:
model.fit(tmp_x,fr_token_pad,batch_size=1024,epochs=30,verbose=1,validation_split=0.2)

Train on 110288 samples, validate on 27573 samples
Epoch 1/30
110288/110288 [==============================] - 15s 138us/step - loss: 3.1043 - val_loss: 2.6636
Epoch 2/30
110288/110288 [==============================] - 13s 116us/step - loss: 2.2545 - val_loss: 1.9271
Epoch 3/30
110288/110288 [==============================] - 13s 116us/step - loss: 1.7347 - val_loss: 1.5842
Epoch 4/30
110288/110288 [==============================] - 13s 116us/step - loss: 1.4869 - val_loss: 1.3944
Epoch 5/30
110288/110288 [==============================] - 13s 116us/step - loss: 1.3414 - val_loss: 1.2799
Epoch 6/30
110288/110288 [==============================] - 13s 117us/step - loss: 1.2311 - val_loss: 1.1639
Epoch 7/30
110288/110288 [==============================] - 13s 116us/step - loss: 1.1057 - val_loss: 1.0478
Epoch 8/30
110288/110288 [==============================] - 13s 116us/step - loss: 1.0191 - val_loss: 0.9701
Epoch 9/30
110288/110288 [==============================] - 13s 116us/step - 

In [114]:
pred=model.predict(tmp_x)

In [115]:
pred=np.argmax(pred,axis=-1)
pred=pred.reshape((-1,1))


In [116]:
acc=accuracy_score(pred,fr_token_pad1)

In [117]:
acc

0.9440893018191892

In [118]:
text1=["i liked lemon and grape"]
text1=engtokenizer.texts_to_sequences(text1)
a=pad_sequences(text1,maxlen=max_fr_length,padding='post')
a=a.reshape((-1,21))
pred_of_a=model.predict(a)
pred=np.argmax(pred_of_a,axis=2)

In [119]:
" ".join([ind_w[i] for i in pred[0]])

"j'aime le pamplemousse pamplemousse pamplemousse <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>"